# 新聞爬蟲器

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import time, os, re
import feedparser
import schedule

## 爬蟲檔名

In [65]:
csv_file_tw = 'news_tw.csv'
csv_file_us = 'news_us.csv'

## 台灣新聞

In [ ]:
# Yahoo財經台灣
def crawler_tw():
    #clear_output(wait=True)
    global csv_file_tw
    url1 = 'https://tw.finance.yahoo.com/tw-market'
    url2 = 'https://tw.finance.yahoo.com/news'

    # 載入網頁
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(url1)

    # 模擬滾動到最底
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待網頁更新
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 取得網頁內容
    html = driver.page_source

    # 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')

    # 找到包含title的元素，並獲取標題及source
    titles1 = soup.find_all('a', class_='Fw(b) Fz(20px) Fz(16px)--mobile Lh(23px) Lh(1.38)--mobile C($c-primary-text)! C($c-active-text)!:h LineClamp(2,46px)!--mobile LineClamp(2,46px)!--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled')
    sources1 = soup.find_all('div', class_='C(#959595) Fz(13px) C($c-secondary-text)! D(ib) Mb(6px)')

    # 關閉瀏覽器
    driver.quit()

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service) 
    driver.get(url2)

    # 模擬滾動到最底
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待網頁更新
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # 取得網頁內容
    html = driver.page_source

    # 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')

    # 找到包含title的元素，並獲取標題及source
    titles2 = soup.find_all('a', class_='Fw(b) Fz(20px) Fz(16px)--mobile Lh(23px) Lh(1.38)--mobile C($c-primary-text)! C($c-active-text)!:h LineClamp(2,46px)!--mobile LineClamp(2,46px)!--sm1024 mega-item-header-link Td(n) C(#0078ff):h C(#000) LineClamp(2,46px) LineClamp(2,38px)--sm1024 not-isInStreamVideoEnabled')
    sources2 = soup.find_all('div', class_='C(#959595) Fz(13px) C($c-secondary-text)! D(ib) Mb(6px)')

    # 關閉瀏覽器
    driver.quit()

    # 建立資料框架
    #csv_file = 'title_raw.csv'
    
    # 檢查檔案是否存在
    if not os.path.exists(csv_file_tw):
        # 若檔案不存在，建立空的資料框並儲存為 CSV 檔案
        empty_df = pd.DataFrame(columns=['crawl_date', 'source_date', 'title', 'source', 'source_time', 'Sentiment_label'])
        empty_df.to_csv(csv_file_tw, index=False, encoding='utf-8-sig')
    df_tw = pd.read_csv(csv_file_tw, encoding='utf-8-sig')

    # 逐個獲取並添加爬取的資料至資料框架
    for title, source in zip(titles1, sources1):
        title_text = title.text.strip()
        source_text = source.text.strip()
        date = datetime.now().strftime('%Y/%m/%d %H:%M')

        # 分割「source」中的資料到「source_time」
        source_parts = source_text.split('•')
        if len(source_parts) > 1:
            source_text = source_parts[0].strip()
            source_time = source_parts[1].strip()
        else:
            source_time = None

        # 添加資料至資料框架
        new_data = pd.DataFrame({'crawl_date': [date], 'title': [title_text], 'source': [source_text], 'source_time': [source_time]})
        df_tw = pd.concat([df_tw, new_data], ignore_index=True)
 
    for title, source in zip(titles2, sources2):
        title_text = title.text.strip()
        source_text = source.text.strip()
        date = datetime.now().strftime('%Y/%m/%d %H:%M')

        # 分割「source」中的資料到「source_date」
        source_parts = source_text.split('•')
        if len(source_parts) > 1:
            source_text = source_parts[0].strip()
            source_time = source_parts[1].strip()
        else:
            source_time = None

        # 添加資料至資料框架
        new_data = pd.DataFrame({'crawl_date': [date], 'title': [title_text], 'source': [source_text], 'source_time': [source_time]})
        df_tw = pd.concat([df_tw, new_data], ignore_index=True)
         
    # 將資料框架輸出為 CSV 檔案
    df_tw.to_csv(csv_file_tw, index=False, encoding='utf-8-sig')

    print(f'爬蟲資料已儲存至{csv_file_tw}檔案中。')

In [19]:
def source_date_tw(csvfile):
    clear_output(wait=True)
    global csv_file_tw
    df_tw = pd.read_csv(csvfile)
    # df_tw = pd.read_csv(csvfile, usecols=['crawl_date', 'source_date', 'title', 'source', 'source_time'])
    # 逐行處理資料
    for index, row in df_tw.iterrows():
        crawl_date = datetime.strptime(row['crawl_date'], '%Y/%m/%d %H:%M')
        source_time = row['source_time']

        # 判斷source_time的類型
        if '秒' in source_time:
            sec = int(source_time.split(' ')[0])
            source_date = crawl_date - timedelta(seconds=sec)
        elif '小時' in source_time:
            hr = int(source_time.split(' ')[0])
            source_date = crawl_date - timedelta(hours=hr)
        elif '分鐘' in source_time:
            min = int(source_time.split(' ')[0])
            source_date = crawl_date - timedelta(minutes=min)
        elif source_time == '昨天':
            source_date = crawl_date - timedelta(days=1)
        elif source_time == '前天':
            source_date = crawl_date - timedelta(days=2)
        else:
            days_ago = int(source_time.replace('天前', ''))
            source_date = crawl_date - timedelta(days=days_ago)

        # 將計算得到的source_date填入「source_date」欄位
        df_tw.at[index, 'source_date'] = source_date.strftime('%Y/%m/%d')

    # 儲存處理後的資料至 CSV 檔案
    df_tw.to_csv(csvfile, index=False, encoding='utf-8-sig')
    # 回傳處理後的資料框
    return df_tw

In [20]:
def remove_symbols_tw(csvfile):
    clear_output(wait=True)
    global csv_file_tw
    df_tw = pd.read_csv(csvfile)

    # 逐行處理資料
    for index, row in df_tw.iterrows():
        title = row['title']
        if '【公告】' in title:
            df_tw = df_tw.drop(index)  # 刪除含有【公告】的列
        else:
            clean_title = re.sub(r'【.*?】|《.*?》', '', title)
            df_tw.at[index, 'title'] = clean_title

    # 檢查重複的title並保留最後一筆
    df_tw = df_tw.drop_duplicates(subset='title', keep='last')
    #df.drop('source_time', axis=1, inplace=True)
    df_tw = df_tw[['crawl_date', 'source_date', 'title', 'source', 'source_time', 'Sentiment_label']]

    # 按舊至新重新排序
    df_tw = df_tw.sort_values('source_date')

    # 儲存處理後的資料至 CSV 檔案
    df_tw.to_csv(csvfile, index=False, encoding='utf-8-sig')

    # 回傳處理後的資料框
    return df_tw

In [21]:
crawler_tw()
df_tw = source_date_tw(csv_file_tw)
df_tw = remove_symbols_tw(csv_file_tw)

爬蟲資料已儲存至news_tw.csv檔案中。


## 美國新聞

In [67]:
def crawler_us(csv_file_us="news_us.csv"):

    # Yahoo Finance RSS 來源(一般爬蟲方法被阻擋)
    feeds = [
        "https://finance.yahoo.com/rss/topstories",
        "https://finance.yahoo.com/news/rss"
    ]

    all_entries = []

    for feed_url in feeds:
        feed = feedparser.parse(feed_url)
        all_entries.extend(feed.entries)

    if not all_entries:
        print("⚠️ 沒有抓到任何新聞。")
        return

    # 構建資料框架
    crawl_time = datetime.now().strftime("%Y/%m/%d %H:%M")
    df = pd.DataFrame([{
        "crawl_date": crawl_time,
        "source_time": e.published if 'published' in e else None,
        "title": e.title,
        "source": "Yahoo Finance RSS",
        "Sentiment_label": "",
        "Positive_sentim": "",
        "Negative_sentim": "",
        "Neutral_sentim": "",
        "link": e.link
    } for e in all_entries])

    # 若 CSV 存在，讀取原資料並合併
    if os.path.exists(csv_file_us):
        df_old = pd.read_csv(csv_file_us, encoding="utf-8-sig")
        df = pd.concat([df_old, df], ignore_index=True)

    # 移除重複標題
    df.drop_duplicates(subset=["title"], inplace=True)

    # 輸出 CSV
    df.to_csv(csv_file_us, index=False, encoding="utf-8-sig")
    print(f"✅ 爬蟲資料已儲存至 {csv_file_us}，共 {len(df)} 筆新聞。")

In [70]:
def source_date_us(csvfile):
    
    df_us = pd.read_csv(csvfile)

    for index, row in df_us.iterrows():
        crawl_date = datetime.strptime(row['crawl_date'], '%Y/%m/%d %H:%M')
        source_time = str(row['source_time']).strip()

        try:
            # RSS ISO 日期格式
            source_date = datetime.strptime(source_time, "%Y-%m-%dT%H:%M:%SZ")
        except (ValueError, TypeError):
            # 原本「幾秒前/分鐘/小時/天前」處理
            if 'second ago' in source_time or 'seconds ago' in source_time:
                sec = int(re.search(r'\d+', source_time).group())
                source_date = crawl_date - timedelta(seconds=sec)
            elif 'minute ago' in source_time or 'minutes ago' in source_time:
                min = int(re.search(r'\d+', source_time).group())
                source_date = crawl_date - timedelta(minutes=min)
            elif 'hour ago' in source_time or 'hours ago' in source_time:
                hr = int(re.search(r'\d+', source_time).group())
                source_date = crawl_date - timedelta(hours=hr)
            elif source_time == 'yesterday':
                source_date = crawl_date - timedelta(days=1)
            else:
                # fallback: 天數
                m = re.search(r'\d+', source_time)
                days_ago = int(m.group()) if m else 0
                source_date = crawl_date - timedelta(days=days_ago)

        # 填入 source_date
        df_us.at[index, 'source_date'] = source_date.strftime('%Y/%m/%d')

    # 清理標題重複
    df_us['title'] = df_us['title'].str.replace("‘|’", "'", regex=True)
    df_us['title'] = df_us['title'].str.replace("—", "-", regex=True)
    df_us = df_us.drop_duplicates(subset='title', keep='last')

    # 保留欄位順序
    df_us = df_us[['crawl_date', 'source_date', 'title', 'source', 'source_time', 
                   'Sentiment_label', 'Positive_sentim', 'Negative_sentim', 'Neutral_sentim']]

    # 按日期排序
    df_us = df_us.sort_values('source_date')

    # 儲存 CSV
    df_us.to_csv(csvfile, index=False, encoding='utf-8-sig')
    return df_us


## 排程器

In [71]:
crawler_us()
df_us = source_date_us(csv_file_us)

✅ 爬蟲資料已儲存至 news_us.csv，共 49 筆新聞。


In [25]:
# def job():
#     clear_output(wait=True)
#     crawler_tw()
#     df_tw = source_date_tw(csv_file_tw)
#     df_tw = remove_symbols_tw(csv_file_tw)
#     crawler_us()
#     df_us = source_date_us(csv_file_us)
#     print(f"新聞爬蟲完成時間：{datetime.now().strftime('%Y/%m/%d %H:%M')}")

# schedule.every().day.at("14:00").do(job)

# while True:
#     schedule.run_pending()
#     time.sleep(1)